In [1]:
# Import libraries
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
# Check virtual environment: should be: '/Users/James/anaconda3/envs/mimic/bin/python'
sys.executable

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
# Set up paths
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_folder = os.path.join(project_root, 'src')

In [4]:
# Import src functions
sys.path.insert(0, src_folder)
from modeling import *
from model_access import *
from s3_storage import *

In [5]:
# Import data
train = from_s3(bucket='mimic-jamesi',
               filename='acute_kidney_failure_train.csv',
               index_col=0)

test = from_s3(bucket='mimic-jamesi',
               filename='acute_kidney_failure_test.csv',
               index_col=0)
print('--> Importing done')

--> Importing done


In [6]:
X_train, X_test, y_train, y_test, feature_names = final_cleaning(ids = ['subject_id', 'hadm_id'],
                                                                 target ='target', train=train, test=test)
print('--> Cleaning done')

--> Cleaning done


In [7]:
def run_final_test(model, X_test, y_test):
    m = open_pickle(model)
    predict = m.predict_proba(X_test)
    score = roc_auc_score(y_test, predict[:,-1])
    print("{} score: ".format(model), score)
    return score

## Logistic Regression

In [8]:
logistic_score = run_final_test('logistic_regression', X_test, y_test)

logistic_regression score:  0.8461391485176046


## Decision Tree

In [9]:
# dt_score = run_final_test('decision_tree', X_test, y_test)

## Random Forest

In [10]:
# rf_score = run_final_test('random_forest', X_test, y_test)

## LightGBM

In [11]:
lightgbm_score = run_final_test('light_gbm', X_test, y_test)

light_gbm score:  0.8871510341057569


/Users/James/anaconda3/envs/mimic/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


## Neural Network

In [12]:
nn_score = run_final_test('neural_network', X_test, y_test)

Using TensorFlow backend.


neural_network score:  0.8743742211498353
